In [1]:
!pip3 install -q datasets peft accelerate evaluate scikit-learn bitsandbytes rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependenc

In [2]:
!pip3 install -q --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 112.5 MB/s eta 0:00:00


In [3]:
import torch, torch.nn as nn, random, numpy as np, evaluate
from datasets import load_dataset
from peft import PeftModel
from torch.nn.utils.rnn import pad_sequence
import tqdm
from transformers import GPT2Model
from transformers import (
    GPT2TokenizerFast, GPT2Model, GPT2LMHeadModel,
    TrainingArguments, Trainer,BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.metrics import classification_report

device = (
    "mps" if torch.backends.mps.is_available() else
    "cuda" if torch.cuda.is_available() else "cpu"
)

SEED = 42
torch.manual_seed(SEED); np.random.seed(SEED); random.seed(SEED)

tok = GPT2TokenizerFast.from_pretrained("gpt2")
tok.pad_token = tok.eos_token

2025-05-21 16:35:34.286782: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747845334.467451      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747845334.520348      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [4]:
def make_qlora_model(base_name, head_cls, lora_r=8, task_type="SEQ_CLS"):
    qconf = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16,
    )

    if head_cls is GPT2LMHeadModel:
        base = GPT2LMHeadModel.from_pretrained(
            base_name, quantization_config=qconf, device_map="auto"
        )
    else:
        base = GPT2Model.from_pretrained(
            base_name, quantization_config=qconf, device_map="auto"
        )
    model = head_cls(base) if head_cls is not GPT2LMHeadModel else base

    lora_cfg = LoraConfig(
        r=lora_r,
        lora_alpha=16,
        lora_dropout=0.05,
        task_type=getattr(TaskType, task_type),
        target_modules=["c_attn"],
    )
    model = get_peft_model(model, lora_cfg).to(device)
    return model


In [5]:
class GPT2Sentiment(nn.Module):
    def __init__(self, base, n_labels=2):
        super().__init__()
        self.base = base
        self.config = base.config
        self.dropout = nn.Dropout(0.1)
        self.cls = nn.Linear(base.config.hidden_size, n_labels)
    def forward(self, input_ids=None, attention_mask=None,
                labels=None, **kwargs):
        kwargs.pop("num_items_in_batch", None)
        hs = self.base(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs
        ).last_hidden_state

        logits = self.cls(self.dropout(hs[:, -1]))
        loss = None
        if labels is not None:
            loss = nn.functional.cross_entropy(logits, labels)
        return {"loss": loss, "logits": logits}

sent_model = make_qlora_model("gpt2", GPT2Sentiment, lora_r=8, task_type="SEQ_CLS")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [6]:
def hr(t): print("\n" + "═"*15 + " " + t + " " + "═"*15)

hr("Training Sentiment")
raw = load_dataset("amazon_polarity")

def tok_sent(ex):
    return tok(ex["title"] + " " + ex["content"],
               truncation=True, max_length=256)

train_s = raw["train"].shuffle(seed=SEED).select(range(200000)).map(tok_sent)
val_s   = raw["test"] .select(range(20000)).map(tok_sent)
train_s = train_s.rename_column("label", "labels")
val_s   = val_s.rename_column("label", "labels")


═══════════════ Training Sentiment ═══════════════


README.md:   0%|          | 0.00/6.81k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/260M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/258M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/255M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [7]:
args_s = TrainingArguments(
    output_dir="sent-qlora",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    max_grad_norm=0.1,
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    report_to=[]
)

Trainer(sent_model, args_s,
        train_dataset=train_s,
        eval_dataset=val_s,
        tokenizer=tok).train()
sent_model.save_pretrained("sent-qlora")
base = GPT2Model.from_pretrained("gpt2")

sent_adapter = PeftModel.from_pretrained(base, "sent-qlora")

/tmp/ipykernel_19/560513592.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  Trainer(sent_model, args_s,
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,1.100000,0.161651
2,0.607300,0.153367
3,0.560600,0.148000


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.h.0.attn.c_attn.lora_A.default.weight', 'base_model.model.h.0.attn.c_attn.lora_B.default.weight', 'base_model.model.h.1.attn.c_attn.lora_A.default.weight', 'base_model.model.h.1.attn.c_attn.lora_B.default.weight', 'base_model.model.h.2.attn.c_attn.lora_A.default.weight', 'base_model.model.h.2.attn.c_attn.lora_B.default.weight', 'base_model.model.h.3.attn.c_attn.lora_A.default.weight', 'base_model.model.h.3.attn.c_attn.lora_B.default.weight', 'base_model.model.h.4.attn.c_attn.lora_A.default.weight', 'base_model.model.h.4.attn.c_attn.lora_B.default.weight', 'base_model.model.h.5.attn.c_attn.lora_A.default.weight', 'base_model.model.h.5.attn.

In [8]:
hr("Evaluation")

sent_val = load_dataset("amazon_polarity", split="test[:20000]")
preds, refs = [], []
for ex in tqdm.tqdm(sent_val, desc="Sentiment"):
    enc = tok(ex["title"] + " " + ex["content"],
              return_tensors="pt", truncation=True, max_length=256).to(device)
    logits = sent_model(**enc)["logits"]
    preds.append(int(logits.argmax(-1)))
    refs.append(ex["label"])

acc = evaluate.load("accuracy").compute(predictions=preds, references=refs)["accuracy"]
f1  = evaluate.load("f1").compute(predictions=preds, references=refs)["f1"]
print(f"Sentiment –  Acc: {acc:.4f}   F1: {f1:.4f}")
print("Classification Report:")
print(classification_report(refs, preds, target_names=["negative","positive"]))



═══════════════ Evaluation ═══════════════


Sentiment: 100%|██████████| 20000/20000 [08:36<00:00, 38.76it/s]


Sentiment –  Acc: 0.9496   F1: 0.9506
Classification Report:
              precision    recall  f1-score   support

    negative       0.95      0.95      0.95      9786
    positive       0.95      0.95      0.95     10214

    accuracy                           0.95     20000
   macro avg       0.95      0.95      0.95     20000
weighted avg       0.95      0.95      0.95     20000

